In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn


pd.set_option('display.max_columns', 30)

### ここだけ手動で設定

In [103]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]
phase_name_list = ["rest", "practice", "boredom", "flow", "ultra", "overload"]
target_columns = ["bpm", "ibi", "lf", "hf","lf/hf","FC3","FC4","FCz","mean_3ch"]
target_phases = ["boredom", "flow","ultra","overload"]

# データセットの整形

In [3]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

In [42]:
import statsmodels.api as sm

### フェーズごとの分析

In [115]:
for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):
        if "kim" in path:
            continue
        df = pd.read_csv(path,index_col=0).fillna(0)
        # column
        # row
        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0:
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)


    
    export_data = {
        
        "target": target_columns,
        "linear-p": [],
        "linear-rsquared": [],
        "linear-rsquared_adj": [],
        "linear-coef": [],
        "nonlinear-p": [],
        "nonlinear-rsquared": [],
        "nonlinear-rsquared_adj": [],
        "nonlinear-coef": [],
        
    }

    for target_column in target_columns:
        


        x = flow_dataset[target_column]
        y = flow_dataset[["questionnaire_average"]]
        
        
        
        for k in range(2):
            if k==0:
                # 線形単回帰
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["linear-p"].append(results.pvalues[target_column])
                export_data["linear-rsquared"].append(results.rsquared)
                export_data["linear-rsquared_adj"].append(results.rsquared_adj)
                export_data["linear-coef"].append(results.params[target_column])
            else:
                # ２次単回帰
                x = x**2
                
                X = sm.add_constant(x)
                model = sm.OLS(y, X)
                results = model.fit()

                export_data["nonlinear-p"].append(results.pvalues[target_column])
                export_data["nonlinear-rsquared"].append(results.rsquared)
                export_data["nonlinear-rsquared_adj"].append(results.rsquared_adj)
                export_data["nonlinear-coef"].append(results.params[target_column])
                



        

    print(target_phase)

    
    df = pd.DataFrame(export_data)
    print(df)
    
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/regression_results/{i}_{target_phase}_regression.csv"
    df.to_csv(save_path)  
    
    
    
    
# def branchFormulasOnNonlinearOrlinear(i):
#     if i == 0:
        
    

boredom
     target  linear-p  linear-rsquared  linear-rsquared_adj  linear-coef  \
0       bpm  0.722303         0.048351            -0.268866    -0.023489   
1       ibi  0.664789         0.071002            -0.238664     0.002643   
2        lf  0.791162         0.027149            -0.297134    -0.000292   
3        hf  0.400595         0.241401            -0.011466     0.001025   
4     lf/hf  0.340874         0.298338             0.064451    -0.172612   
5       FC3  0.314522         0.326222             0.101629    -5.469243   
6       FC4  0.246629         0.407220             0.209626    -5.021478   
7       FCz  0.669641         0.068911            -0.241451    -2.200552   
8  mean_3ch  0.372154         0.267489             0.023319    -4.663941   

   nonlinear-p  nonlinear-rsquared  nonlinear-rsquared_adj  nonlinear-coef  
0     0.750851            0.038794               -0.281608   -1.386871e-04  
1     0.637115            0.083572               -0.221904    1.753724e-06  


# 被験者ごとの分析

https://atmarkit.itmedia.co.jp/ait/articles/2109/14/news024.html

# 線形重回帰分析（マルチこに気をつけよう）

In [12]:
x = flow_dataset[["bpm", "ibi","mean_3ch"]]
y = flow_dataset[["questionnaire_average"]]

#全要素が1の列を説明変数の先頭に追加,切片をつけるために必ず必要
X = sm.add_constant(x)
 
#モデルの設定
model = sm.OLS(y, X)
 
#回帰分析の実行
results = model.fit()
 
#結果の詳細を表示
print(results.summary())

## いい結果のように思えるが、多重共線性により偽の有意性が出てしまっている

                              OLS Regression Results                             
Dep. Variable:     questionnaire_average   R-squared:                       0.977
Model:                               OLS   Adj. R-squared:                  0.942
Method:                    Least Squares   F-statistic:                     28.05
Date:                   Thu, 12 Jan 2023   Prob (F-statistic):             0.0346
Time:                           23:00:04   Log-Likelihood:                 6.7720
No. Observations:                      6   AIC:                            -5.544
Df Residuals:                          2   BIC:                            -6.377
Df Model:                              3                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        112.4892 

/Users/miyakooti/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


# 非線形（２次）単回帰分析

In [70]:
import statsmodels.formula.api as smf

In [94]:
x = flow_dataset[["mean_3ch"]]
y = flow_dataset[["questionnaire_average"]]

# X = np.column_stack((x, x**2))
x = x+x**2

#全要素が1の列を説明変数の先頭に追加（絶対必要！！）
X = sm.add_constant(x)
 
#モデルの設定
model = sm.OLS(y, X)
 
#回帰分析の実行
results = model.fit()
 
#結果の詳細を表示
print(results.summary())

                              OLS Regression Results                             
Dep. Variable:     questionnaire_average   R-squared:                       0.248
Model:                               OLS   Adj. R-squared:                  0.060
Method:                    Least Squares   F-statistic:                     1.318
Date:                   Fri, 13 Jan 2023   Prob (F-statistic):              0.315
Time:                           21:13:35   Log-Likelihood:                -3.6624
No. Observations:                      6   AIC:                             11.32
Df Residuals:                          4   BIC:                             10.91
Df Model:                              1                                         
Covariance Type:               nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.2259 

/Users/miyakooti/opt/anaconda3/lib/python3.9/site-packages/statsmodels/stats/stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [51]:
X

,const,lf
flow,1.0,227.008970
flow,1.0,227.008970
flow,1.0,479.902818
flow,1.0,445.662097
flow,1.0,428.892008
flow,1.0,449.168624


### 参考
- https://takacity.blog.fc2.com/blog-entry-305.html
- https://self-development.info/%E3%80%90%E5%88%9D%E5%BF%83%E8%80%85%E8%84%B1%E5%87%BA%E3%80%91statsmodels%E3%81%AB%E3%82%88%E3%82%8B%E9%87%8D%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90%E7%B5%90%E6%9E%9C%E3%81%AE%E8%A6%8B%E6%96%B9/
- https://teratail.com/questions/256310

In [59]:
# 決定係数
# 0.9以上	非常によい
# 0.7以上0.9未満	よい
# 0.5以上0.7未満	あまりよくない
# 0.5未満	悪い

# Dep. Variable:     questionnaire_average   R-squared:                       0.915（決定係数。説明変数が目的変数をどれくらい説明できるか）
# Model:                               OLS   Adj. R-squared:                  0.893（自由度調整済み決定係数）
# Method:                    Least Squares   F-statistic:                     42.82（F値）
# Date:                   Fri, 13 Jan 2023   Prob (F-statistic):            0.00282（F値の現れる確率）
# Time:                           12:17:03   Log-Likelihood:                 2.8630
# No. Observations:           データの行数   AIC:                            -1.726
# Df Residuals:               残差の自由度   BIC:                            -2.142
# Df Model:                   要因の自由度                                        
# Covariance Type:  nonrobust（変数間の相関関係）


#       coef（回帰係数。傾き）    std err（標準誤差）  t      P>|t|      [0.025  0.975]
# ------------------------------------------------------------------------------
# const          7.3625      0.174     42.246      0.000       6.879       7.846
# lf         -6.729e-06   1.03e-06     -6.544      0.003   -9.58e-06   -3.87e-06
# ==============================================================================
# Omnibus:                          nan   Durbin-Watson:                   1.765
# Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.156
# Skew:                          -0.179   Prob(JB):                        0.925
# Kurtosis:                       2.296   Cond. No.                     3.93e+05
# ==============================================================================

In [60]:
dir(results)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abat_diagonal',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_is_nested',
 '_use_t',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_prediction',
 'get_robustcov_results',
 'info_criteria',


In [111]:
print(results.__doc__)


    Results class for for an OLS model.

    Parameters
    ----------
    model : RegressionModel
        The regression model instance.
    params : ndarray
        The estimated parameters.
    normalized_cov_params : ndarray
        The normalized covariance parameters.
    scale : float
        The estimated scale of the residuals.
    cov_type : str
        The covariance estimator used in the results.
    cov_kwds : dict
        Additional keywords used in the covariance specification.
    use_t : bool
        Flag indicating to use the Student's t in inference.
    **kwargs
        Additional keyword arguments used to initialize the results.

    See Also
    --------
    RegressionResults
        Results store for WLS and GLW models.

    Notes
    -----
    Most of the methods and attributes are inherited from RegressionResults.
    The special methods that are only available for OLS are:

    - get_influence
    - outlier_test
    - el_test
    - conf_int_el
    


In [107]:
dir(results.params)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__redu

- https://www.statsmodels.org/stable/gettingstarted.html
- https://www.statsmodels.org/stable/api.html